In [13]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from PIL import Image
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

In [2]:
dataset_train = load_dataset('unsloth/LaTeX_OCR', split='train[:3000]')

In [3]:
type(dataset_train)

datasets.arrow_dataset.Dataset

In [4]:
dataset_train.column_names

['image', 'text']

In [5]:
def custom_collate_fn(batch):
    texts = []
    images = []
    for example in batch:
        image_inputs = process_vision_info(example["messages"])
        text = processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        )
        texts.append(text.strip())
        images.append(image_inputs)

    # Tokenize the texts and process the images
    new_batch = processor(text=texts, images=images, return_tensors="pt", padding=True)

    # The labels are the input_ids, and we mask the padding tokens and image tokens in the loss computation
    labels = new_batch["input_ids"].clone()

    # Mask image tokens
    image_token_id = [
        processor.tokenizer.convert_tokens_to_ids(
            processor.tokenizer.special_tokens_map["boi_token"]
        )
    ]

    # Mask tokens for not being used in the loss computation
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100


In [23]:
processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")

In [24]:
instruction = 'Convert the equation images to LaTeX equations.'


def convert_to_conversation(sample):
    conversation = [
        # User Prompt : 유저의 질문
        { 'role': 'user',
          'content' : [
            {'type' : 'text',  'text'  : instruction},
            {'type' : 'image', 'image' : sample['image']} ]
        },

        # Assistant Response : 모델이 하는 리턴
        { 'role' : 'assistant',
          'content' : [
            {'type' : 'text',  'text'  : sample['text']} ]
        },
    ]
    return { 'messages' : conversation }

In [16]:
def process_vision_info(messages: list[dict]) -> list[Image.Image]:
    """
    데이터셋 이미지를 RGB 포맷으로 변경
    """
    image_inputs = []
    for msg in messages:
        content = msg.get("content", [])
        if not isinstance(content, list):
            content = [content]
 
        for element in content:
            if isinstance(element, dict) and ("image" in element or element.get("type") == "image"):
                image = element["image"]
                image_inputs.append(image.convert("RGB"))
    return image_inputs

In [17]:
from tqdm import tqdm

In [18]:
train_dataset = [convert_to_conversation(sample) for sample in tqdm(dataset_train, total=len(dataset_train))]

100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 3206.53it/s]


In [19]:
type(train_dataset)

list

In [25]:
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)

In [26]:
for batch in dataloader:
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.PngImagePlugin.PngImageFile'>

In [18]:
batch

NameError: name 'batch' is not defined

In [40]:
texts = []
images = []

In [48]:
def process_vision_info(messages: list[dict]) -> list[Image.Image]:
    # print(messages)
    image_inputs = []
    for msg in messages:
        content = msg.get("content", [])
        # print(content)
        if not isinstance(content, list):
            content = [content]
 
        for element in content:
            if isinstance(element, dict) and ("image" in element or element.get("type") == "image"):
                image = element["image"]
                # print(image)
                image_inputs.append(image.convert("RGB"))
    return image_inputs

In [30]:
processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [31]:
type(processor)

transformers.models.gemma3.processing_gemma3.Gemma3Processor

In [56]:
for example in batch:
    image_inputs = process_vision_info(example["messages"]) # PIL.Image.Image
    text = processor.apply_chat_template(
        example["messages"], add_generation_prompt=False, tokenize=False
    )
    prin(text)

NameError: name 'examples' is not defined